In [78]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import boston_valuation as val
import pandas as pd
import numpy as np

In [40]:
data = pd.read_csv('boston.csv')

In [41]:
target = np.log(data['PRICE'])
features = data.drop(['PRICE', 'INDUS', 'AGE', 'Unnamed: 0'], axis=1)
features


,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [42]:
property_stats = np.ndarray(shape=(1, 11))
property_stats = features.mean().values.reshape(1, 11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [43]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)
MSE = mean_squared_error(target, fitted_vals)
print('Rsquared',r2_score(target, fitted_vals))
RMSE = np.sqrt(mean_squared_error(target, fitted_vals))

Rsquared 0.7891431041340249


In [44]:
def get_log_estimate(nr_rooms, students_per_classroom, 
                    next_to_river=False, high_confidence=True):
    property_stats[0][4] = nr_rooms
    property_stats[0][8] = students_per_classroom
    log_estimate = regr.predict(property_stats)[0]
    
    if next_to_river:
        property_stats[0][2] = 1
    else:
        property_stats[0][2] = 0
    
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    
    return log_estimate, upper_bound, lower_bound, interval


In [45]:
zillow_median_price = 583.3
scale_factor = zillow_median_price / np.e**(np.median(target))
scale_factor

27.514150943396224

In [60]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval."""
    
    
    if rm < 1 or ptratio < 1:
        print("That is unrealistic. Please, try again.")
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, ptratio, next_to_river=chas, 
                                                   high_confidence=large_range)
    dollar_est = np.e**log_est *1000*scale_factor
    dollar_high = np.e**upper *1000*scale_factor
    dollar_low = np.e**lower *1000*scale_factor
    
    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)
    print(f"The estimated property value is {rounded_est}$ \nAt 95% confidence the valuation range is {rounded_high}$ and {rounded_low}$")

In [61]:
get_dollar_estimate(rm=0, ptratio=30, chas=True)

That is unrealistic. Please, try again.


In [80]:
val.get_dollar_estimate(rm=5,ptratio=30, chas=True, large_range=False)

The estimated property value is 365000.0$ 
At 68 confidence the valuation range is 440000.0$ and 302000.0$


C:\Users\Eid josef\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
